# Komplexitätsmaße

In den vorigen Lektionen haben wir schon mit Tokens pro Satz hantiert. Je länger ein Satz ist, desto schwerer ist er verständlich.

Allerdings gibt es noch weit mehr Größen, die du dafür ermitteln kannst. Die werden wir uns jetzt genauer ansehen.

## Daten einladen

Wie gewohnt lädst du zunächst die linguistisch analysierten Daten aus der Datenbank:

In [ ]:
!pip install textacy
!python -m spacy download de_core_news_lg

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles \
                   WHERE datePublished<'2021-01-01' \
                   ORDER BY datePublished", sql, index_col="id", parse_dates=["datePublished"])

## Nochmal Tokens pro Satz berechnen

Zum Verleich berechnest du nochmal die Anzahl der Tokens pro Satz und lässt dir das Ergebnis als Histogramm anzeigen:

In [ ]:
df["token_per_sentence"] = df["no_tokens"] / df["no_sentences"]
df[["token_per_sentence"]].plot.hist()

Wie du sehen kannst, ist die Bandbreite ganz erheblich. Die meisten Artikel liegen zwischen 8 Tokens pro Satz und 20 Tokens pro Satz. Das wollen wir uns nun genauer anschauen. 

## Komplexitätsmaße berechnen

Zunächst brauchst du wieder `spacy` und das geeignete Modell.

In [ ]:
import spacy
nlp = spacy.load("de_core_news_lg")

Über `textacy` kannst du nun die Komplexitätsmaße berechnen. Der Durchlauf braucht ungefähr 10 Minuten:

In [ ]:
import textacy.text_stats
from tqdm.auto import tqdm

df["fulltext"] = df["title"] + "\n" + df["header"] + "\n" + df["text"]
for i, r in tqdm(df.iterrows(), total=len(df)):
    tdoc = textacy.make_spacy_doc(r["fulltext"], lang="de_core_news_lg")
    ts = textacy.text_stats.TextStats(tdoc)
    df.at[i, "entropy"] = ts.entropy
    df.at[i, "coleman_liau_index"] = ts.coleman_liau_index
    df.at[i, "gunning_fog_index"] = ts.gunning_fog_index
    df.at[i, "flesch_kincaid_grade_level"] = ts.flesch_kincaid_grade_level
    df.at[i, "smog_index"] = ts.smog_index
    df.at[i, "wiener_sachtextformel"] = ts.wiener_sachtextformel

Genauer Informationen zu den Metriken findest du unter https://textacy.readthedocs.io/en/0.10.1/api_reference/misc.html#textacy.text_stats.api.TextStats

## Entropy

Die Entropie des Textes, d.h. umso höher, je mehr Durcheinander in den Wörtern

In [ ]:
df[["entropy"]].plot.hist()

## Coleman Liau Index

Lesbarkeitsindex, der die Jahre der Schulbildung zählt, die für ein Verständnis notwendig sind. Ähnlich zu `flesch_kincaid_grade_level()` und `smog_index()`, allerdings werden Zeichen pro Wort statt Silben verwendet.

In [ ]:
df[["coleman_liau_index"]].plot.hist()

## Smog Index

Lesbarkeitstest aus medizinischer Literatur und Healthcare. Auch hier werden die Jahre an Schulbildung abgeschätzt, die zum Verständnis notwendig sind. Ähnlich zu `flesch_kincaid_grade_level()` und auch als Ersatz für `gunning_fog_index()` gedacht.


In [ ]:
df[["smog_index"]].plot.hist()

## Flesch Kincaid Grade

Einer der beliebtesten Lesbarkeitsindizes, weil er ganz generell und sprachunabhänig verwendet werden kann. Schätzt ebenfalls die Anzahl der Schuljahre zum Verständnis ab.

In [ ]:
df[["flesch_kincaid_grade_level"]].plot.hist()

## Wiener Sachtextformel

Diese Index ist spezifisch für deutsche Sprache konstruiert und wird in deutschsprachiger Literatur überwiegend verwendet. Auch hier wird wieder die Anzahl der zum Verständnis notwendigen Schuljahre abgeschätzt-.

In [ ]:
df[["wiener_sachtextformel"]].plot.hist()

## Schreibstil von Autoren

Es ist interessant, die Komplexität der Texte unterschiedliche Autoren zu vergleichen. Im ersten Schritt bestimmst du dazu die Top-20-Autoren

In [ ]:
top_authors = df.groupby("author").agg({"url": "count"}).sort_values("url").tail(20).index.values
top_author_articles = df[df["author"].isin(top_authors)]

Wie schon in der letzten Lektion gezeigt, berechnen wir zunächst den Median der Wiener Sachtextformel-Werte und ordnen die Labels entsprechend. Statt eines Boxplots verwenden wir einen sog. *Violin-Plot*, der noch etwas mehr Information beinhaltet. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 10))
labels = top_author_articles.groupby("author").agg({"wiener_sachtextformel": "median"}).sort_values("wiener_sachtextformel").index.values
sns.violinplot(y="author", x="wiener_sachtextformel", data=top_author_articles, order=labels, palette="viridis")

In der Auswertung kannst du erkennen, dass *Daniel AJ Sokolov* zwar relativ gut lesbar schreibt, aber einige Ausreißer hat, die schwierig zu lesen sind. Gleiches gilt für *Alexander Neumann*. *Olivia von Westernhagen* deckt ein großes Spektrum ab ebenso wie *Bernd Mewes*.

Du kannst dir noch die am schwierigsten zu lesenden Artikel ausgeben:

In [ ]:
pd.set_option("display.max_colwidth", None)
df.sort_values("wiener_sachtextformel").tail(10)[["author", "title", "url"]]

Du kannst nun selbst die Artikel aufrufen und überlegen, ob die wirklich besonders schwer lesbar sind.

## Lesbarkeit enthält viel Heuristik

Die Formeln zur Berechnung der Lesbarkeitswerte sind relativ kompliziert und sehr heuristisch. Ob der Index im Einzelfall wirklich die Komplexität abbildet, kannst du selbst überprüfen - es wird nicht immer stimmen. Über die Statistik und die große Menge an Daten gleicht sich das im Allgemeinen wieder aus. Du solltest nur im Hinterkopf behalten, dass es sich um *Phänomenologie* handelt.